In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [13]:
#Import Data Set
dataset = pd.read_csv('Covid19_LineList_Records.csv')
pd.set_option('display.max_columns', None)
dataset

,Unnamed: 0,age,gender,caseConfirmationDate,livesInGroundZero,traveledToGroundZero,recordSource,lineListSource,hasTravelHistory,id,location.id,locationType,groundZeroExposure,notes,relevantTravelHistoryLocation,sequenceAvailable,symptomStartDate,hospitalAdmissionDate,relevantTravelHistoryDates,outcome,didDie,didRecover,outcomeDate,symptoms,ageRange,chronicDisease
0,0,27.0,female,2020-04-04T00:00:00Z,False,False,https://www.prensalibre.com/guatemala/comunita...,OPEN,False,00002e80-d790-4d77-aa02-7da86338c41b,GuatemalaCity_GuatemalaCity_Guatemala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,2020-03-26T00:00:00Z,False,False,NaN,OPEN,False,00004cec-61fe-43a0-b4c4-cbb2a2c4e932,NewcastleuponTyneTyneandWear_England_UnitedKin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,2020-03-25T00:00:00Z,False,False,https://coronavirus.health.ny.gov/county-count...,OPEN,False,00009339-ab52-4de7-afad-3a8ca35f700e,NewYork_UnitedStates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,2020-03-22T00:00:00Z,False,False,https://coronavirus.health.ny.gov/county-count...,OPEN,False,0000ef82-6365-4ee6-bb44-00b8575dbf3c,NewYorkCity_NewYork_UnitedStates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,2020-03-29T00:00:00Z,False,False,https://www.mercurynews.com/2020/03/20/map-cor...,OPEN,False,000174f4-8b80-4832-b7d5-333e4a3ccfcf,California_UnitedStates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266866,866,NaN,NaN,2020-03-23T00:00:00Z,False,False,https://www.rki.de/DE/Content/InfAZ/N/Neuartig...,OPEN,False,ffff04d1-239a-4e3c-a983-7c927382b157,Bavaria_Germany,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266867,867,NaN,NaN,2020-02-06T00:00:00Z,False,False,http://wjw.hunan.gov.cn/wjw/xxgk/gzdt/zyxw_1/2...,OPEN,False,ffff0ebf-9d94-43b7-ada2-688bb84057fd,YongzhouCity_Hunan_China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266868,868,NaN,NaN,2020-03-25T00:00:00Z,False,False,https://www.nj.gov/health/cd/topics/covid2019_...,OPEN,False,ffff15af-e211-441b-aded-1442599c936a,NewJersey_UnitedStates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266869,869,NaN,NaN,2020-02-01T00:00:00Z,False,False,https://bnonews.com/index.php/2020/01/the-late...,OPEN,False,ffff1bb3-7bf8-4dcc-bda5-fa632a8f7d91,Beijing_China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
patient_level_nulls = dataset[["age", "gender", "symptoms", "hospitalAdmissionDate", "chronicDisease", "outcome", "didDie"]]
patient_level = patient_level_nulls.dropna(subset = ["didDie"])
patient_level

,age,gender,symptoms,hospitalAdmissionDate,chronicDisease,outcome,didDie
183,43.0,male,NaN,2020-01-26T00:00:00Z,NaN,discharged,False
875,48.0,female,NaN,2020-02-03T00:00:00Z,NaN,discharge,False
925,58.0,female,NaN,2020-02-18T00:00:00Z,NaN,discharge,False
931,NaN,NaN,NaN,NaN,NaN,death,True
983,NaN,NaN,NaN,NaN,NaN,stable,False
...,...,...,...,...,...,...,...
265358,65.0,male,acute respiratory distress syndrome:pneumonia,NaN,hypertension:chronic kidney disease,died,True
265423,NaN,NaN,NaN,NaN,NaN,stable,False
265720,57.0,male,cardiogenic shock:acute coronary syndrome:hear...,NaN,hypertension,died,True
265851,NaN,NaN,NaN,NaN,NaN,stable,False


In [30]:
patient_level["chronicDisease"].unique()

array([nan, 'hypertension',
       'hypertension, coronary heart disease, diabetes, Tuberculosis',
       'hypertension:cerebrovascular accident infarct',
       'diabetes:hypertension',
       'hypertension:diabetes:valvular heart disease',
       'diabetes, hypertension', 'hypertension:colon cancer',
       'hypertensive', 'diabetes:hypertenstion:upper git bleeding',
       'prostate hypertrophy',
       "Parkinson's disease for five years, taking medicine of Madopar",
       '"thought to have had other pre-existing conditions"', 'asthma',
       'cardiac disease', 'diabetes:hypertension:chronic kidney disease',
       'hypertension:diabetes',
       'asthma:hypertension:atherosclerosis:coronary artery disease',
       'hypertension:cardiac disease',
       'hypertension:diabetes:chronic kidney disease',
       'hepatitis B, diabetes', 'HIV positive',
       'chronic kidney disease:hypertension:diabetes',
       'hypertension:diabetes:dyslipidemia',
       'hypertension, diabetes, co